In [ ]:
!unzip semeval2024_dev_release.zip

unzip:  cannot find or open semeval2024_dev_release.zip, semeval2024_dev_release.zip.zip or semeval2024_dev_release.zip.ZIP.


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
!pip install accelerate -U

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
import re

def clean_text(text):
    text = re.sub(r'(\\n)+', '. ', text)

    # Remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    # Remove any remaining backslashes
    text = text.replace("\\", " ")

    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch

In [ ]:
!pip install sentencepiece
model_name = "google/t5-v1_1-base"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

In [ ]:
def classify_text(text, labels, model, tokenizer):
    # Construct a prompt that lists all labels and asks to classify the text
    label_list = ', '.join(f'"{label}"' for label in labels)
    text = clean_text(text)
    prompt = f"Classify the following text by the persuasion techniques it uses, selecting from {label_list}: {text}"

    # Encode the prompt
    inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)

    # Generate output sequence
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=512, num_beams=3)

    # Decode and return the result
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result

In [ ]:
# List of persuasion techniques as labels
persuasion_techniques = [
    "Appeal to authority", "Appeal to fear/prejudice", "Bandwagon", "Black-and-white Fallacy/Dictatorship",
    "Causal Oversimplification", "Doubt", "Exaggeration/Minimisation", "Flag-waving",
    "Glittering generalities (Virtue)", "Loaded Language", "Misrepresentation of Someone's Position (Straw Man)",
    "Name calling/Labeling", "Obfuscation, Intentional vagueness, Confusion", "Presenting Irrelevant Data (Red Herring)",
    "Reductio ad hitlerum", "Repetition", "Slogans", "Smears", "Thought-terminating cliché", "Whataboutism"
]

# Sample meme text
sample_meme = "Si dices que estamos en medio de una pandemia mortal, pero aún admites las fronteras abiertas \\ n \\ nyou's un mentiroso o un imbécil completo"

# Classify the meme text
classification_result = classify_text(sample_meme, persuasion_techniques, model, tokenizer)
print("Classification Results:", classification_result)

Classification Results: n't.n't.n't.nnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn wan


In [ ]:
from datasets import Dataset
import pickle
import json


# Example data in your format
# data = [
#     {
#         "id": "63292",
#         "text": "Es por eso que somos libres \\ n \\ ntis es por eso que estamos a salvo \\ n",
#         "labels": ["Causal Oversimplification"]
#     },
#     {
#         "id": "70419",
#         "text": "Si dices que estamos en medio de una pandemia mortal, pero aún admites las fronteras abiertas \\ n \\ nyou's un mentiroso o un imbécil completo",
#         "labels": [
#             "Loaded Language",
#             "Name calling/Labeling",
#             "Black-and-white Fallacy/Dictatorship",
#             "Smears"
#         ]
#     }
# ]

# def convert_to_t5_format(example):
#     labels_str = ', '.join(example['labels'])
#     return {
#         "input_text": f"meme: {clean_text(example['text'])}",
#         "target_text": f"{labels_str}"
#     }
def convert_to_t5_format(example):
    labels_str = ', '.join(example['labels'])

    label_list = [
    "Appeal to authority", "Appeal to fear/prejudice", "Bandwagon", "Black-and-white Fallacy/Dictatorship",
    "Causal Oversimplification", "Doubt", "Exaggeration/Minimisation", "Flag-waving",
    "Glittering generalities (Virtue)", "Loaded Language", "Misrepresentation of Someone's Position (Straw Man)",
    "Name calling/Labeling", "Obfuscation, Intentional vagueness, Confusion", "Presenting Irrelevant Data (Red Herring)",
    "Reductio ad hitlerum", "Repetition", "Slogans", "Smears", "Thought-terminating cliché", "Whataboutism"
    ]

    return {
        "input_text": f"Classify the following text by the persuasion techniques it uses, selecting from {label_list}: and here is meme text: {clean_text(example['text'])}",
        "target_text": f"{labels_str}"
        }



with open('/content/drive/MyDrive/datasets/annotations_v2/semeval2024_dev_release/subtask1/train.json', 'r', encoding='utf-8') as f:
    train_data = json.load(f)

with open('/content/drive/MyDrive/datasets/annotations_v2/semeval2024_dev_release/subtask1/validation.json', 'r', encoding='utf-8') as f:
    validation_data = json.load(f)

# print(data)
# Convert to a Hugging Face dataset
train_dataset = Dataset.from_list(train_data)
validation_dataset = Dataset.from_list(validation_data)

# Function to convert examples to T5 format

# Apply the function
train_dataset = train_dataset.map(convert_to_t5_format)
validation_dataset = validation_dataset.map(convert_to_t5_format)


Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments
import torch

model_name = "t5-base"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)
tokenizer = T5Tokenizer.from_pretrained(model_name)

# Tokenize the data
def tokenize_data(example):
    # Tokenize input text
    model_inputs = tokenizer(
        example['input_text'],
        max_length=512,
        truncation=True,
        padding="max_length",
        return_tensors="pt"  # Ensure output is PyTorch tensors
    ).to(device)  # Move tensors to MPS device

    # Tokenize target text and handle padding
    labels = tokenizer(
        example['target_text'],
        max_length=128,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    ).input_ids.squeeze().to(device)  # Remove unnecessary dimensions and move to device

    # Replace padding token ID with -100 for the labels
    labels = torch.where(labels == tokenizer.pad_token_id, torch.tensor(-100, device=device), labels)

    return {"input_ids": model_inputs.input_ids.squeeze(), "attention_mask": model_inputs.attention_mask.squeeze(), "labels": labels}


train_dataset = train_dataset.map(tokenize_data, batched=True)
validation_dataset = validation_dataset.map(tokenize_data, batched=True)

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=75,
    predict_with_generate=True,
    no_cuda=False
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.563550
2,0.735900,0.526908
3,0.552100,0.539236
4,0.509300,0.530515
5,0.483900,0.510649
6,0.464600,0.502943
7,0.439300,0.501753
8,0.425400,0.509909
9,0.425400,0.505125
10,0.406500,0.511539


TrainOutput(global_step=32850, training_loss=0.23444820479533807, metrics={'train_runtime': 42801.9115, 'train_samples_per_second': 12.266, 'train_steps_per_second': 0.767, 'total_flos': 3.19702892544e+17, 'train_loss': 0.23444820479533807, 'epoch': 75.0})

#####

In [ ]:
import json
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
from datasets import load_dataset

# Assuming model and tokenizer are loaded as per the training script
model_name = "t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)

# Load specific checkpoint after training
model = T5ForConditionalGeneration.from_pretrained("./results/checkpoint-32850")


# model = T5ForConditionalGeneration.from_pretrained(model_name)
model.to(device)

def predict_labels(text, model, tokenizer, device):
    """Generate labels for a given text using the fine-tuned model."""

    label_list = [
    "Appeal to authority", "Appeal to fear/prejudice", "Bandwagon", "Black-and-white Fallacy/Dictatorship",
    "Causal Oversimplification", "Doubt", "Exaggeration/Minimisation", "Flag-waving",
    "Glittering generalities (Virtue)", "Loaded Language", "Misrepresentation of Someone's Position (Straw Man)",
    "Name calling/Labeling", "Obfuscation, Intentional vagueness, Confusion", "Presenting Irrelevant Data (Red Herring)",
    "Reductio ad hitlerum", "Repetition", "Slogans", "Smears", "Thought-terminating cliché", "Whataboutism"
    ]

    input_text = f"Classify the following text by the persuasion techniques it uses, selecting from {label_list}: and here is meme text: {clean_text(text)}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)
    outputs = model.generate(**inputs, max_length=30, num_beams=3)
    predicted_labels = tokenizer.decode(outputs[0], skip_special_tokens=True).split(',')
    predicted_labels = [label.strip() for label in predicted_labels]
    return predicted_labels

def process_predictions(data, model, tokenizer, device):
    """Process the dataset and save predictions in a JSON file."""
    results = []
    for item in data:
        text = item['text']
        cleaned_text = clean_text(text)  # Clean the text as defined earlier
        labels = predict_labels(cleaned_text, model, tokenizer, device)
        results.append({
            "id": item['id'],
            "labels": labels
        })
    return results

# Load your dataset
validation_data = load_dataset('json', data_files={'validation': '/content/drive/MyDrive/datasets/annotations_v2/semeval2024_dev_release/subtask1/dev_unlabeled.json'})['validation']

# Generate predictions
predictions = process_predictions(validation_data, model, tokenizer, device)

# Save predictions to a JSON file
with open('t5_predicted_labels_100.json', 'w') as f:
  json.dump(predictions,f,indent=4)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Generating validation split: 0 examples [00:00, ? examples/s]

In [ ]:
!zip -r /content/file.zip /content/results


  adding: content/results/ (stored 0%)
  adding: content/results/checkpoint-32850/ (stored 0%)
  adding: content/results/checkpoint-32850/spiece.model (deflated 48%)
  adding: content/results/checkpoint-32850/model.safetensors (deflated 9%)
  adding: content/results/checkpoint-32850/special_tokens_map.json (deflated 85%)
  adding: content/results/checkpoint-32850/optimizer.pt (deflated 7%)
  adding: content/results/checkpoint-32850/config.json (deflated 63%)
  adding: content/results/checkpoint-32850/scheduler.pt (deflated 56%)
  adding: content/results/checkpoint-32850/training_args.bin (deflated 51%)
  adding: content/results/checkpoint-32850/added_tokens.json (deflated 83%)
  adding: content/results/checkpoint-32850/generation_config.json (deflated 30%)
  adding: content/results/checkpoint-32850/tokenizer_config.json (deflated 94%)
  adding: content/results/checkpoint-32850/rng_state.pth (deflated 25%)
  adding: content/results/checkpoint-32850/trainer_state.json (deflated 82%)
  ad

In [ ]:
from google.colab import files
files.download("/content/t5trainfile.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/t5trainfile.zip'

Archive:  /content/drive/MyDrive/t5trainfile.zip
   creating: content/results/
   creating: content/results/checkpoint-32850/
  inflating: content/results/checkpoint-32850/spiece.model  
  inflating: content/results/checkpoint-32850/model.safetensors  
  inflating: content/results/checkpoint-32850/special_tokens_map.json  
  inflating: content/results/checkpoint-32850/optimizer.pt  
  inflating: content/results/checkpoint-32850/config.json  
  inflating: content/results/checkpoint-32850/scheduler.pt  
  inflating: content/results/checkpoint-32850/training_args.bin  
  inflating: content/results/checkpoint-32850/added_tokens.json  
  inflating: content/results/checkpoint-32850/generation_config.json  
  inflating: content/results/checkpoint-32850/tokenizer_config.json  
  inflating: content/results/checkpoint-32850/rng_state.pth  
  inflating: content/results/checkpoint-32850/trainer_state.json  
   creating: content/results/checkpoint-32000/
  inflating: content/results/checkpoint-3200

In [ ]:
import json
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
from datasets import load_dataset

# Assuming model and tokenizer are loaded as per the training script
model_name = "t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)

# Load specific checkpoint after training
model = T5ForConditionalGeneration.from_pretrained("/content/content/results/checkpoint-32850")


# model = T5ForConditionalGeneration.from_pretrained(model_name)
model.to(device)

def predict_labels(text, model, tokenizer, device):
    """Generate labels for a given text using the fine-tuned model."""

    label_list = [
    "Appeal to authority", "Appeal to fear/prejudice", "Bandwagon", "Black-and-white Fallacy/Dictatorship",
    "Causal Oversimplification", "Doubt", "Exaggeration/Minimisation", "Flag-waving",
    "Glittering generalities (Virtue)", "Loaded Language", "Misrepresentation of Someone's Position (Straw Man)",
    "Name calling/Labeling", "Obfuscation, Intentional vagueness, Confusion", "Presenting Irrelevant Data (Red Herring)",
    "Reductio ad hitlerum", "Repetition", "Slogans", "Smears", "Thought-terminating cliché", "Whataboutism"
    ]

    input_text = f"Classify the following text by the persuasion techniques it uses, selecting from {label_list}: and here is meme text: {clean_text(text)}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)
    outputs = model.generate(**inputs, max_length=30, num_beams=3)
    predicted_labels = tokenizer.decode(outputs[0], skip_special_tokens=True).split(',')
    predicted_labels = [label.strip() for label in predicted_labels]
    return predicted_labels

def process_predictions(data, model, tokenizer, device):
    """Process the dataset and save predictions in a JSON file."""
    results = []
    for item in data:
        text = item['text']
        cleaned_text = clean_text(text)  # Clean the text as defined earlier
        labels = predict_labels(cleaned_text, model, tokenizer, device)
        results.append({
            "id": item['id'],
            "labels": labels
        })
    return results

# Load your dataset
validation_data = load_dataset('json', data_files={'validation': '/content/drive/MyDrive/datasets/annotations_v2/semeval2024_dev_release/subtask1/dev_unlabeled.json'})['validation']
ar_test_data= load_dataset('json', data_files={'validation': '/content/drive/MyDrive/datasets/test_data/ar_subtask1_test_unlabeled.json'})['validation']
bg_test_data= load_dataset('json', data_files={'validation': '/content/drive/MyDrive/datasets/test_data/bg_subtask1_test_unlabeled.json'})['validation']
mk_test_data= load_dataset('json', data_files={'validation': '/content/drive/MyDrive/datasets/test_data/mk_subtask1_test_unlabeled.json'})['validation']
en_test_data= load_dataset('json', data_files={'validation': '/content/drive/MyDrive/datasets/test_data/en_subtask1_test_unlabeled.json'})['validation']

# Generate predictions
dev_predictions = process_predictions(validation_data, model, tokenizer, device)
ar_predictions = process_predictions(ar_test_data, model, tokenizer, device)
bg_predictions = process_predictions(bg_test_data, model, tokenizer, device)
mk_predictions = process_predictions(mk_test_data, model, tokenizer, device)
en_predictions = process_predictions(en_test_data, model, tokenizer, device)
# Save predictions to a JSON file
with open('dev_t5predicted_labels_75.json', 'w') as f:
  json.dump(dev_predictions,f,indent=4)

with open('ar_t5predicted_labels_75.json', 'w') as f:
  json.dump(ar_predictions,f,indent=4)

with open('bg_t5predicted_labels_75.json', 'w') as f:
  json.dump(bg_predictions,f,indent=4)

with open('mk_t5predicted_labels_75.json', 'w') as f:
  json.dump(mk_predictions,f,indent=4)

with open('en_t5predicted_labels_75.json', 'w') as f:
  json.dump(en_predictions,f,indent=4)

print("Predictions saved to JSON files.")



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Generating validation split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Predictions saved to JSON files.


In [ ]:
!pip install sklearn_hierarchical_classification


  Preparing metadata (setup.py) ... done
  Created wheel for sklearn_hierarchical_classification: filename=sklearn_hierarchical_classification-1.3.2-py3-none-any.whl size=16912 sha256=e3ed47662a2274ec9fe9bb5949fd720ce01e9b4a1d27270b002d031530d0e206
  Stored in directory: /root/.cache/pip/wheels/87/09/e3/1619c1a173b533be37bda720b5f5aaa719e116d66ff7c01a12
Successfully built sklearn_hierarchical_classification


In [ ]:
!python /content/drive/MyDrive/subtask_1_2a.py -g /content/drive/MyDrive/datasets/dev_gold_labels/dev_subtask1_en.json -p /content/dev_t5predicted_labels_75.json

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) ['', 'Appeal to fear/', 'Appeal to fear/prejud', 'Black-and-', 'Black-and-white Fallacy/', 'Black-and-white Fallacy/Dictator', 'Confusion', 'Flag-w', 'G', 'Glittering general', 'Glittering generalities (Vir', 'Glittering generalities (Virt', 'Glittering generalities (Virtu', 'Glittering generalities (Virtue', 'Intentional vagueness', "Misrepresentation of Someone's Position (", 'Name calling', 'Name calling/Lab', 'Name calling/Labe', 'Obfuscation', 'S', 'Sm', 'Smear'] will be ignored
  warnings.warn(
f1_h=0.58140	prec_h=0.59122	rec_h=0.57190
